In [2]:
import datetime as dt
import pandas as pd
import requests
import folium
import json

#print('importado')

In [3]:
boundaries_map = 'static/res/custom.geo.json'

In [4]:
#FUNDOS DE PREVIDÊNCIA COMO PORCENTAGEM DO PIB
df_share_gdp = pd.read_csv('static/res/2.1. Assets as share of GDP.csv')

#Eliminando colunas inúteis
df_share_gdp.drop('Pension Plan Type', axis=1, inplace=True)
df_share_gdp.drop('PPTYP', axis=1, inplace=True)
df_share_gdp.drop('DTYP', axis=1, inplace=True)
df_share_gdp.drop('Definition Type', axis=1, inplace=True)
df_share_gdp.drop('CTYP', axis=1, inplace=True)
df_share_gdp.drop('Contract Type', axis=1, inplace=True)
df_share_gdp.drop('VAR', axis=1, inplace=True)
df_share_gdp.drop('Variable', axis=1, inplace=True)
df_share_gdp.drop('IND', axis=1, inplace=True)
df_share_gdp.drop('Indicator', axis=1, inplace=True)
df_share_gdp.drop('Flags', axis=1, inplace=True)
df_share_gdp.drop('Flag Codes', axis=1, inplace=True)
df_share_gdp.drop('Reference Period', axis=1, inplace=True)
df_share_gdp.drop('Reference Period Code', axis=1, inplace=True)
df_share_gdp.drop('PowerCode', axis=1, inplace=True)
df_share_gdp.drop('PowerCode Code', axis=1, inplace=True)
df_share_gdp.drop('Unit', axis=1, inplace=True)
df_share_gdp.drop('YEA', axis=1, inplace=True)

#Dividindo dataframes por anos
df_share_gdp_2008=df_share_gdp[df_share_gdp['Year'] == 2008]
df_share_gdp_2008.reset_index(inplace=True, drop=True)
df_share_gdp_2009=df_share_gdp[df_share_gdp['Year'] == 2009]
df_share_gdp_2009.reset_index(inplace=True, drop=True)
df_share_gdp_2010=df_share_gdp[df_share_gdp['Year'] == 2010]
df_share_gdp_2010.reset_index(inplace=True, drop=True)
df_share_gdp_2011=df_share_gdp[df_share_gdp['Year'] == 2011]
df_share_gdp_2011.reset_index(inplace=True, drop=True)
df_share_gdp_2012=df_share_gdp[df_share_gdp['Year'] == 2012]
df_share_gdp_2012.reset_index(inplace=True, drop=True)
df_share_gdp_2013=df_share_gdp[df_share_gdp['Year'] == 2013]
df_share_gdp_2013.reset_index(inplace=True, drop=True)
df_share_gdp_2014=df_share_gdp[df_share_gdp['Year'] == 2014]
df_share_gdp_2014.reset_index(inplace=True, drop=True)
df_share_gdp_2015=df_share_gdp[df_share_gdp['Year'] == 2015]
df_share_gdp_2015.reset_index(inplace=True, drop=True)
df_share_gdp_2016=df_share_gdp[df_share_gdp['Year'] == 2016]
df_share_gdp_2016.reset_index(inplace=True, drop=True)
df_share_gdp_2017=df_share_gdp[df_share_gdp['Year'] == 2017]
df_share_gdp_2017.reset_index(inplace=True, drop=True)
df_share_gdp_2018=df_share_gdp[df_share_gdp['Year'] == 2018]
df_share_gdp_2018.reset_index(inplace=True, drop=True)

df_share_gdp_2018.head(10)

,COU,Country,Year,Unit Code,Value
0,AUS,Australia,2018,PC,137.864
1,AUT,Austria,2018,PC,5.550
2,BEL,Belgium,2018,PC,6.826
3,CAN,Canada,2018,PC,84.390
4,CZE,Czech Republic,2018,PC,9.171
5,DNK,Denmark,2018,PC,45.271
6,FIN,Finland,2018,PC,49.740
7,FRA,France,2018,PC,0.705
8,DEU,Germany,2018,PC,6.901
9,GRC,Greece,2018,PC,0.749


In [5]:
#GASTOS COM PREVIDÊNCIA COM PORCENTAGEM DO PIB
df_gdp_spending = pd.read_csv('static/res/1.1. OECD Pensions as share GDP.csv', header=0)

#Eliminando colunas inúteis
df_gdp_spending.drop('INDICATOR', axis=1, inplace=True)
df_gdp_spending.drop('MEASURE', axis=1, inplace=True)
df_gdp_spending.drop('FREQUENCY', axis=1, inplace=True)
df_gdp_spending.drop('Flag Codes', axis=1, inplace=True)

#Dropa linhas de gasto privado
i = 0
for gasto in df_gdp_spending['SUBJECT']:
    if gasto == 'PRIV':
        df_gdp_spending.drop([i], axis=0, inplace=True)

    i+=1
df_gdp_spending.reset_index(inplace=True,drop=True)

#Adicionando dados brasileiros
bra_data = {'LOCATION': ['BRA','BRA','BRA','BRA','BRA','BRA','BRA','BRA','BRA','BRA','BRA','BRA','BRA','BRA','BRA','BRA','BRA','BRA','BRA','BRA','BRA','BRA'], 
            'SUBJECT': ['PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB'], 
            'TIME': [1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016], 
            'Value': [4.98,5.25,5.15,5.76,5.75,5.77,6.00,6.08,6.52,6.65,7.00,7.20,7.04,6.78,7.16,7.38,8.20,8.30,8.40,8.50,9.00,9.90]}

#Adicionando dados russos
rus_data = {'LOCATION': ['RUS','RUS','RUS'], 
            'SUBJECT': ['PUB','PUB','PUB'], 
            'TIME': [2014,2015,2016], 
            'Value': [7.05,8.94,9.10]}

#Adicionando valores máximos e mínimos
max_data = {'LOCATION': ['MAX','MAX','MAX','MAX','MAX','MAX','MAX','MAX','MAX','MAX','MAX','MAX','MAX','MAX','MAX','MAX','MAX','MAX','MAX','MAX','MAX','MAX','MAX'], 
            'SUBJECT': ['PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB'], 
            'TIME': [1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016], 
            'Value': [20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20]}

min_data = {'LOCATION': ['MIN','MIN','MIN','MIN','MIN','MIN','MIN','MIN','MIN','MIN','MIN','MIN','MIN','MIN','MIN','MIN','MIN','MIN','MIN','MIN','MIN','MIN','MIN'], 
            'SUBJECT': ['PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB','PUB'], 
            'TIME': [1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016], 
            'Value': [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]}

df_bra = pd.DataFrame(bra_data)
df_rus = pd.DataFrame(rus_data)
df_max = pd.DataFrame(max_data)
df_min = pd.DataFrame(min_data)
df_gdp_spending = df_gdp_spending.append(df_bra)
df_gdp_spending = df_gdp_spending.append(df_rus)
df_gdp_spending = df_gdp_spending.append(df_max)
df_gdp_spending = df_gdp_spending.append(df_min)
df_gdp_spending.reset_index(inplace=True,drop=True)

#Dividindo dataframes por anos
df_gdp_spending_1994=df_gdp_spending[df_gdp_spending['TIME'] == 1994]
df_gdp_spending_1994.reset_index(inplace=True, drop=True)
df_gdp_spending_1995=df_gdp_spending[df_gdp_spending['TIME'] == 1995]
df_gdp_spending_1995.reset_index(inplace=True, drop=True)
df_gdp_spending_1996=df_gdp_spending[df_gdp_spending['TIME'] == 1996]
df_gdp_spending_1996.reset_index(inplace=True, drop=True)
df_gdp_spending_1997=df_gdp_spending[df_gdp_spending['TIME'] == 1997]
df_gdp_spending_1997.reset_index(inplace=True, drop=True)
df_gdp_spending_1998=df_gdp_spending[df_gdp_spending['TIME'] == 1998]
df_gdp_spending_1998.reset_index(inplace=True, drop=True)
df_gdp_spending_1999=df_gdp_spending[df_gdp_spending['TIME'] == 1999]
df_gdp_spending_1999.reset_index(inplace=True, drop=True)
df_gdp_spending_2000=df_gdp_spending[df_gdp_spending['TIME'] == 2000]
df_gdp_spending_2000.reset_index(inplace=True, drop=True)
df_gdp_spending_2001=df_gdp_spending[df_gdp_spending['TIME'] == 2001]
df_gdp_spending_2001.reset_index(inplace=True, drop=True)
df_gdp_spending_2002=df_gdp_spending[df_gdp_spending['TIME'] == 2002]
df_gdp_spending_2002.reset_index(inplace=True, drop=True)
df_gdp_spending_2003=df_gdp_spending[df_gdp_spending['TIME'] == 2003]
df_gdp_spending_2003.reset_index(inplace=True, drop=True)
df_gdp_spending_2004=df_gdp_spending[df_gdp_spending['TIME'] == 2004]
df_gdp_spending_2004.reset_index(inplace=True, drop=True)
df_gdp_spending_2005=df_gdp_spending[df_gdp_spending['TIME'] == 2005]
df_gdp_spending_2005.reset_index(inplace=True, drop=True)
df_gdp_spending_2006=df_gdp_spending[df_gdp_spending['TIME'] == 2006]
df_gdp_spending_2006.reset_index(inplace=True, drop=True)
df_gdp_spending_2007=df_gdp_spending[df_gdp_spending['TIME'] == 2007]
df_gdp_spending_2007.reset_index(inplace=True, drop=True)
df_gdp_spending_2008=df_gdp_spending[df_gdp_spending['TIME'] == 2008]
df_gdp_spending_2008.reset_index(inplace=True, drop=True)
df_gdp_spending_2009=df_gdp_spending[df_gdp_spending['TIME'] == 2009]
df_gdp_spending_2009.reset_index(inplace=True, drop=True)
df_gdp_spending_2010=df_gdp_spending[df_gdp_spending['TIME'] == 2010]
df_gdp_spending_2010.reset_index(inplace=True, drop=True)
df_gdp_spending_2011=df_gdp_spending[df_gdp_spending['TIME'] == 2011]
df_gdp_spending_2011.reset_index(inplace=True, drop=True)
df_gdp_spending_2012=df_gdp_spending[df_gdp_spending['TIME'] == 2012]
df_gdp_spending_2012.reset_index(inplace=True, drop=True)
df_gdp_spending_2013=df_gdp_spending[df_gdp_spending['TIME'] == 2013]
df_gdp_spending_2013.reset_index(inplace=True, drop=True)
df_gdp_spending_2014=df_gdp_spending[df_gdp_spending['TIME'] == 2014]
df_gdp_spending_2014.reset_index(inplace=True, drop=True)
df_gdp_spending_2015=df_gdp_spending[df_gdp_spending['TIME'] == 2015]
df_gdp_spending_2015.reset_index(inplace=True, drop=True)
df_gdp_spending_2016=df_gdp_spending[df_gdp_spending['TIME'] == 2016]
df_gdp_spending_2016.reset_index(inplace=True, drop=True)


#df_gdp_spending_2015.head()
df_gdp_spending.tail(50)

,LOCATION,SUBJECT,TIME,Value
1185,BRA,PUB,2016,9.90
1186,RUS,PUB,2014,7.05
1187,RUS,PUB,2015,8.94
1188,RUS,PUB,2016,9.10
1189,MAX,PUB,1994,20.00
1190,MAX,PUB,1995,20.00
1191,MAX,PUB,1996,20.00
1192,MAX,PUB,1997,20.00
1193,MAX,PUB,1998,20.00
1194,MAX,PUB,1999,20.00


In [6]:
a = open('static/res/custom.geo.json')
a = json.load(a)
i=0

c = []

for co in a['features']:
    c.append(a['features'][i]['properties']['admin'])
    print(a['features'][i]['properties']['admin'])
    
    i+=1

The Bahamas
Belize
Canada
Costa Rica
Cuba
Dominican Republic
Guatemala
Honduras
Haiti
Jamaica
Greenland
Mexico
Nicaragua
Panama
El Salvador
Puerto Rico
Trinidad and Tobago
United States of America
Argentina
Bolivia
Chile
Colombia
Ecuador
Falkland Islands
Guyana
Peru
Brazil
Suriname
Uruguay
Venezuela
Afghanistan
United Arab Emirates
Paraguay
Bangladesh
Brunei
Armenia
Azerbaijan
China
Northern Cyprus
Cyprus
Bhutan
Georgia
Indonesia
India
Iran
Iraq
Israel
Jordan
Japan
Kyrgyzstan
Cambodia
South Korea
Kuwait
Kazakhstan
Myanmar
Mongolia
Laos
Lebanon
Sri Lanka
Malaysia
Nepal
Pakistan
Oman
North Korea
Palestine
Qatar
Philippines
Saudi Arabia
Syria
Thailand
Tajikistan
Turkmenistan
East Timor
Taiwan
Turkey
Uzbekistan
Vietnam
Yemen
Angola
Burundi
Benin
Burkina Faso
Botswana
Central African Republic
Ivory Coast
Cameroon
Democratic Republic of the Congo
Republic of Congo
Djibouti
Algeria
Egypt
Eritrea
Ethiopia
Gabon
Gambia
Ghana
Guinea
Guinea Bissau
Equatorial Guinea
Kenya
Liberia
Libya
Lesotho
Mor

In [7]:
#Lista com todos os países do geojson
countr_dict = {'Country': c}
countr_dict['Country']

['The Bahamas',
 'Belize',
 'Canada',
 'Costa Rica',
 'Cuba',
 'Dominican Republic',
 'Guatemala',
 'Honduras',
 'Haiti',
 'Jamaica',
 'Greenland',
 'Mexico',
 'Nicaragua',
 'Panama',
 'El Salvador',
 'Puerto Rico',
 'Trinidad and Tobago',
 'United States of America',
 'Argentina',
 'Bolivia',
 'Chile',
 'Colombia',
 'Ecuador',
 'Falkland Islands',
 'Guyana',
 'Peru',
 'Brazil',
 'Suriname',
 'Uruguay',
 'Venezuela',
 'Afghanistan',
 'United Arab Emirates',
 'Paraguay',
 'Bangladesh',
 'Brunei',
 'Armenia',
 'Azerbaijan',
 'China',
 'Northern Cyprus',
 'Cyprus',
 'Bhutan',
 'Georgia',
 'Indonesia',
 'India',
 'Iran',
 'Iraq',
 'Israel',
 'Jordan',
 'Japan',
 'Kyrgyzstan',
 'Cambodia',
 'South Korea',
 'Kuwait',
 'Kazakhstan',
 'Myanmar',
 'Mongolia',
 'Laos',
 'Lebanon',
 'Sri Lanka',
 'Malaysia',
 'Nepal',
 'Pakistan',
 'Oman',
 'North Korea',
 'Palestine',
 'Qatar',
 'Philippines',
 'Saudi Arabia',
 'Syria',
 'Thailand',
 'Tajikistan',
 'Turkmenistan',
 'East Timor',
 'Taiwan',
 'Tur

In [8]:
#Dicionário com todos os DFs de gasto. Serve para seleção dos dados desejados.
data_df = {1994: df_gdp_spending_1994, 
           1995: df_gdp_spending_1995, 
           1996: df_gdp_spending_1996, 
           1997: df_gdp_spending_1997, 
           1998: df_gdp_spending_1998, 
           1999: df_gdp_spending_1999, 
           2000: df_gdp_spending_2000,
           2001: df_gdp_spending_2001, 
           2002: df_gdp_spending_2002, 
           2003: df_gdp_spending_2003, 
           2004: df_gdp_spending_2004, 
           2005: df_gdp_spending_2005, 
           2006: df_gdp_spending_2006, 
           2007: df_gdp_spending_2007, 
           2008: df_gdp_spending_2008, 
           2009: df_gdp_spending_2009, 
           2010: df_gdp_spending_2010, 
           2011: df_gdp_spending_2011, 
           2012: df_gdp_spending_2012, 
           2013: df_gdp_spending_2013, 
           2014: df_gdp_spending_2014, 
           2015: df_gdp_spending_2015, 
           2016: df_gdp_spending_2016}
data_df

{1994:    LOCATION SUBJECT  TIME   Value
 0       AUS     PUB  1994   3.164
 1       AUT     PUB  1994  12.195
 2       BEL     PUB  1994   9.438
 3       CAN     PUB  1994   4.652
 4       CZE     PUB  1994   5.539
 5       DNK     PUB  1994   7.348
 6       FIN     PUB  1994   8.904
 7       FRA     PUB  1994  11.562
 8       DEU     PUB  1994  10.014
 9       GRC     PUB  1994   9.066
 10      ISL     PUB  1994   2.380
 11      IRL     PUB  1994   4.626
 12      ITA     PUB  1994  13.175
 13      JPN     PUB  1994   5.561
 14      KOR     PUB  1994   1.066
 15      LUX     PUB  1994   8.221
 16      MEX     PUB  1994   0.582
 17      NLD     PUB  1994   5.562
 18      NZL     PUB  1994   5.881
 19      NOR     PUB  1994   5.546
 20      POL     PUB  1994   9.408
 21      PRT     PUB  1994   6.452
 22      ESP     PUB  1994   8.652
 23      SWE     PUB  1994   8.219
 24      CHE     PUB  1994   5.911
 25      TUR     PUB  1994   0.551
 26      GBR     PUB  1994   4.957
 27      USA  

In [12]:
#Mapa coroplético
map2 = folium.Map(location=[30,5], zoom_start=1.8, min_zoom=1.8, max_bounds=True)

ano = 2014 #ano desejado

folium.Choropleth(
    geo_data=boundaries_map,
    data=data_df[ano], #seleciona os dados por meio do dicionário de dataframes
    columns=['LOCATION', 'Value'],
    key_on='feature.properties.adm0_a3',
    label_color='White',
    fill_color='BuPu',
    fill_opacity=0.8,
    line_color='White',
    line_opacity=0.4,
    legend_color='White'
).add_to(map2)

map2.save(outfile='templates/map.html')


print("Total de países com dados: " + str(len(data_df[ano])-2)) #-2 pq 2 valores são de max e min
map2

Total de países com dados: 39


In [10]:
a

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'scalerank': 1,
    'featurecla': 'Admin-0 country',
    'labelrank': 4,
    'sovereignt': 'The Bahamas',
    'sov_a3': 'BHS',
    'adm0_dif': 0,
    'level': 2,
    'type': 'Sovereign country',
    'admin': 'The Bahamas',
    'adm0_a3': 'BHS',
    'geou_dif': 0,
    'geounit': 'The Bahamas',
    'gu_a3': 'BHS',
    'su_dif': 0,
    'subunit': 'The Bahamas',
    'su_a3': 'BHS',
    'brk_diff': 0,
    'name': 'Bahamas',
    'name_long': 'Bahamas',
    'brk_a3': 'BHS',
    'brk_name': 'Bahamas',
    'brk_group': None,
    'abbrev': 'Bhs.',
    'postal': 'BS',
    'formal_en': 'Commonwealth of the Bahamas',
    'formal_fr': None,
    'note_adm0': None,
    'note_brk': None,
    'name_sort': 'Bahamas, The',
    'name_alt': None,
    'mapcolor7': 1,
    'mapcolor8': 1,
    'mapcolor9': 2,
    'mapcolor13': 5,
    'pop_est': 309156,
    'gdp_md_est': 9093,
    'pop_year': -99,
    'lastcensus': 2010,
    'gdp_y